# Houses info
<!-- ![landscape of faculty houses](landscape.png) -->
<img src="landscape.png" alt="landscape" width="600"/>
load data and solar data, same data in a year/month (2019),
house 212, 213, 214, 215 (205, 206, 208) pick these 4 houses to replace the sample data,
only load data is needed (w.r.t the form of sample data, house ID needed)

In [1]:
import os
import pandas as pd
import numpy as np

import logging.config, datetime
logger = logging.getLogger(__name__)

import matplotlib.pyplot as plt
%matplotlib inline

In [25]:
house212_2019 = '/Users/Huang/Documents/DQNBattery/data/house212_2019_sift'
house213_2019 = '/Users/Huang/Documents/DQNBattery/data/house213_2019_sift'
house214_2019 = '/Users/Huang/Documents/DQNBattery/data/house214_2019_sift'
house215_2019 = '/Users/Huang/Documents/DQNBattery/data/house215_2019_sift'

house205_2019 = '/Users/Huang/Documents/DQNBattery/data/house205_2019_sift'
house206_2019 = '/Users/Huang/Documents/DQNBattery/data/house206_2019_sift'
house208_2019 = '/Users/Huang/Documents/DQNBattery/data/house208_2019_sift'

# weather 
weather_2019 = '/Users/Huang/Documents/DQNBattery/data/2019_hillside_weather'

In [26]:
def get_file_house(foldername, houseID):
    
    file_list = []  # file name stored in file_list
   
    for root, dirs, files in os.walk(foldername):
        for file in files:
            dirname, filename = os.path.split(file)

#             print(filename.split('2019', 1)[0])
            if filename.split('_', 1)[0] == houseID:
                filename = filename.split('_', 1)[1]        
                file_list.append(filename.split('.', 1)[0])

    file_list.sort()
    
    return file_list

In [27]:
def get_file_weather(foldername, houseID):
    
    file_list = []  # file name stored in file_list
   
    for root, dirs, files in os.walk(foldername):
        for file in files:
            dirname, filename = os.path.split(file)
            
#             print(filename)
#             print(filename.split('2019', 1)[0])      

            if filename.split('2019', 1)[0] == houseID:
                filename = filename.split('weather', 1)[1]        
                file_list.append(filename.split('.', 1)[0])

    file_list.sort()
    
    return file_list

In [28]:
house212_files = get_file_house(house212_2019, "house212")
house213_files = get_file_house(house213_2019, "house213")
house214_files = get_file_house(house214_2019, "house214")
house215_files = get_file_house(house215_2019, "house215")

weather_files = get_file_weather(weather_2019, "weather")

print("weather:", len(weather_files), "\n",
      "house212", len(house212_files), "\n",
      "house213", len(house213_files), "\n",
      "house214", len(house214_files), "\n",
      "house215", len(house215_files))

weather: 334 
 house212 344 
 house213 346 
 house214 334 
 house215 345


# Get the common date files

In [29]:
common = list(set(weather_files)&set(house212_files)&set(house213_files)&set(house214_files)&set(house215_files))
print(len(common))

332


In [31]:
common.sort()
# print(common)

# generate APIS data

In [ ]:
def gener_apis(houseID, common_list, inputfile_dir, outputfile_dir, outputfile_name):
    # join all common dates data into one single file for each node
    
    df = pd.read_csv(inputfile_dir + '/' + houseID + common_list[0] + ".csv")

    df.to_csv(outputfile_dir + '/' + outputfile_name, index=False)

    for i in range(1, len(common_list)):
        df = pd.read_csv(inputfile_dir + '/' + houseID + common_list[i] + ".csv", error_bad_lines=False)
        df.to_csv(outputfile_dir + '/' + outputfile_name, index=False, header=False, mode='a+')


In [ ]:
common_list = common

house212_inputfile_dir = '/Users/Huang/Documents/DQNBattery/data/house212_2019_sift'
house212_outputfile_dir = '/Users/Huang/Documents/DQNBattery/data'
house212_outputfile_name = 'house212_2019_apis_all.csv'

house212_apis = gener_apis("house212_", common, house212_inputfile_dir, house212_outputfile_dir, house212_outputfile_name)


In [ ]:
house213_inputfile_dir = '/Users/Huang/Documents/DQNBattery/data/house213_2019_sift'
house213_outputfile_dir = '/Users/Huang/Documents/DQNBattery/data'
house213_outputfile_name = 'house213_2019_apis_all.csv'

house213_apis = gener_apis("house213_", common, house213_inputfile_dir, house213_outputfile_dir, house213_outputfile_name)


In [ ]:
house214_inputfile_dir = '/Users/Huang/Documents/DQNBattery/data/house214_2019_sift'
house214_outputfile_dir = '/Users/Huang/Documents/DQNBattery/data'
house214_outputfile_name = 'house214_2019_apis_all.csv'

house214_apis = gener_apis("house214_", common, house214_inputfile_dir, house214_outputfile_dir, house214_outputfile_name)


In [ ]:
house215_inputfile_dir = '/Users/Huang/Documents/DQNBattery/data/house215_2019_sift'
house215_outputfile_dir = '/Users/Huang/Documents/DQNBattery/data'
house215_outputfile_name = 'house215_2019_apis_all.csv'

house215_apis = gener_apis("house215_", common, house215_inputfile_dir, house215_outputfile_dir, house215_outputfile_name)


In [ ]:
weather_inputfile_dir = '/Users/Huang/Documents/DQNBattery/data/2019_hillside_weather'
weather_outputfile_dir = '/Users/Huang/Documents/DQNBattery/data'
weather_outputfile_name = 'weather_2019_apis_all.csv'

weather_apis = gener_apis("weather", common, weather_inputfile_dir, weather_outputfile_dir, weather_outputfile_name)


# take average value for every 30 mins

In [ ]:
def timeframe(timestep, inputdata):
    # select the timestamp with every timestep mins
    # slice data for every 15 minutes
    # raw data recorded for each 30s, ~ slice data for every timestep*2 rows
    step = np.arange(0, len(inputdata), timestep * 2)
    outputdata = inputdata.loc[step, :]
    outputdata.index = range(len(outputdata))  # do not keep original index
    return outputdata

In [ ]:
def timeframe_avg(timestep, inputdata):
    inputdata = inputdata.fillna(method='ffill', inplace=False)
    avg = inputdata.groupby(inputdata.index // (timestep * 2)).mean()
    timestamp = timeframe(timestep, inputdata)
    time = timestamp[['timestamp']]
    outputdata = pd.concat((time, avg), axis=1)
    return outputdata

In [ ]:
timestep = 30

house212_ess_dir = '/Users/Huang/Documents/DQNBattery/data/house212_2019_apis_all.csv'
house212_inputdata = pd.read_csv(house212_ess_dir)
house212_outputdata = timeframe_avg(timestep, house212_inputdata)
house212_outputdata.to_csv('/Users/Huang/Documents/APIS/apis-emulator/data/input/Sample/house212_2019_avg.csv', index=False)


In [ ]:
house213_ess_dir = '/Users/Huang/Documents/DQNBattery/data/house213_2019_apis_all.csv'
house213_inputdata = pd.read_csv(house213_ess_dir)
house213_outputdata = timeframe_avg(timestep, house213_inputdata)
house213_outputdata.to_csv('/Users/Huang/Documents/APIS/apis-emulator/data/input/Sample/house213_2019_avg.csv', index=False)


In [ ]:
house214_ess_dir = '/Users/Huang/Documents/DQNBattery/data/house214_2019_apis_all.csv'
house214_inputdata = pd.read_csv(house214_ess_dir)
house214_outputdata = timeframe_avg(timestep, house214_inputdata)
house214_outputdata.to_csv('/Users/Huang/Documents/APIS/apis-emulator/data/input/Sample/house214_2019_avg.csv', index=False)


In [ ]:
house215_ess_dir = '/Users/Huang/Documents/DQNBattery/data/house215_2019_apis_all.csv'
house215_inputdata = pd.read_csv(house215_ess_dir)
house215_outputdata = timeframe_avg(timestep, house215_inputdata)
house215_outputdata.to_csv('/Users/Huang/Documents/APIS/apis-emulator/data/input/Sample/house215_2019_avg.csv', index=False)


In [ ]:
weather_ess_dir = '/Users/Huang/Documents/DQNBattery/data/weather_2019_apis_all.csv'
weather_inputdata = pd.read_csv(weather_ess_dir)
weather_outputdata = timeframe_avg(timestep, weather_inputdata)
weather_outputdata.to_csv('/Users/Huang/Documents/APIS/apis-emulator/data/input/Sample/weather_2019_avg.csv', index=False)


# put all data into one file

In [ ]:
# house 212, 213, 214, 215 four houses

def file_name_csv(input_data, output_data_dir):

    os.chdir(input_data)  # path of original csv data files
    # name = os.path.splitext(raw_data_dir)[0]  # get file name
    input_data = os.getcwd()  # read current file path

    file_list = os.listdir(input_data)

    new_path = output_data_dir
    for path in file_list:
        data = pd.read_csv(path)
        df = pd.DataFrame(data, columns=["timestamp",
                                         "pvc_charge_power"
                                         "ups_output_power",
                                         ])
        # export to csv files
        df.to_csv(os.path.join(new_path, path), index=False)

## leave the data with the same arrangement 

In [ ]:
input_data = pd.read_csv("/Users/Huang/Documents/APIS/apis-emulator/data/input/Sample/house212_2019_avg.csv") 
input_data = input_data.fillna(method="ffill", inplace=False)
df = pd.DataFrame(input_data, columns=["timestamp", "ups_output_power"])

df.rename(columns={"timestamp":"Date"}, inplace=True)
# print(df.Date)
df["Date"] = pd.to_datetime(df.Date)

# df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
# df["Date"] = df["Date"].dt.date

# df["Time"] = df["Date"].dt.time
df["Date"] = df["Date"].dt.strftime('%Y/%m/%d')

display(df)


# df3 = df.unstack(0)
# print(df3)


# new_path = output_data_dir


# for path in file_list:
#     data = pd.read_csv(path)
#     df = pd.DataFrame(data, columns=["timestamp",
#                                      "ups_output_power",
#                                      ])
#     # export to csv files
#     df.to_csv(os.path.join(new_path, path), index=False)

In [ ]:
def old_loadDemand_Sample():
    
    global demand

    demand = {}
    # demand_data = np.genfromtxt('data/input/Sample/sample_load_data.csv', delimiter=',', names=True, dtype=None, encoding='utf-8')
    cols = list(range(6, 52+1, 2))
    cols.insert(0, 1)
    # read column 2, col 7~53 for every 2 cols (1 hour per data point) from input data
    # replace our own data files to the /Sample/ folder
    demand_data = np.loadtxt('cdata/input/Sample/sample_load_data.csv', delimiter=',', skiprows=1, usecols=cols)
    print(demand_data)
    
    for row in demand_data:
        cus_id = "E{0:03d}".format(int(row[0]))
        print("cus_id", cus_id)
        if not demand.get(cus_id):
            demand[cus_id] = []
        demand[cus_id].append(row[1:])
    for cus_id in demand:
        demand[cus_id] = np.array(demand[cus_id])
        gl.displayNames[cus_id] = "Sample_"+cus_id

    return demand

In [ ]:
demandUpdate = old_loadDemand_Sample()


In [ ]:
def old_loadDemand_Sample():
    global demand

    demand = {}
    # demand_data = np.genfromtxt('data/input/Sample/sample_load_data.csv', delimiter=',', names=True, dtype=None, encoding='utf-8')
    cols = list(range(6, 52+1, 2))
    cols.insert(0, 1)
    print(cols)
    # read column 2, col 7~53 for every 2 cols (1 hour per data point) from input data
    # replace our own data files to the /Sample/ folder
    demand_data = np.loadtxt('/Users/Huang/Documents/APIS/apis-emulator/data/input/Sample/sample_load_data.csv', delimiter=',', skiprows=1, usecols=cols)
#     print(demand_data)
    
    for row in demand_data:
        print(int(row[0]), row)
        cus_id = "E{0:03d}".format(int(row[0]))
        print("cus_id", cus_id)
        if not demand.get(cus_id):
            demand[cus_id] = []
        demand[cus_id].append(row[1:])
#         print(demand[cus_id])
        
    print("all id", demand)
    
    for cus_id in demand:
        demand[cus_id] = np.array(demand[cus_id])
        displayNames[cus_id] = "Sample_"+cus_id

    return demand

In [ ]:
displayNames = {}
sample_load = old_loadDemand_Sample()
display(sample_load["E001"])
print(len(sample_load["E001"]))

In [ ]:
def PV_data():  # load house's PV production data
    global pv

    col = 7 #list(range(2, 6, 1)), ups_output_power
#     col.insert(0, 0)
    print(col)
    our_data = np.loadtxt('/Users/Huang/Documents/APIS/apis-emulator/data/input/Sample/house214_2019_avg.csv', delimiter=',', skiprows=1, usecols=col)
    # get the pvc_charge_power column (3rd), house 214, 2019
    # pv = our_data[:, 2]

    pv = {}

    day_len = int(len(our_data) / 48)

    for load_col in our_data:
        house_id = "E{0:03d}".format(int(load_col[0]))
        if not pv.get(house_id):
            pv[house_id] = []

    for day in range(day_len):
        pv[house_id].append(our_data[day * 48:(day + 1) * 48, 2])

    for house_id in pv:
        pv[house_id] = np.array(pv[house_id])

    pv = pv[house_id]

    return pv

In [ ]:
load = PV_data()
print(load)

In [2]:
# weather
def old_pvcUpdate_Sample():
    count_h = float(count_s)/3600

    weight = count_h-int(count_h)
    print("weight: ", weight)
    
    step_now = (int((count_h) / 24)), int((count_h) % 24)
    print("step_now: ", step_now)
    
    step_next = (int((count_h+1)/24)), int((count_h+1) % 24)
    print("step_next:", step_next)
    
    if int(count_h+1) >= sol.size:
        logger.debug("no more solar radiation data")
        return False
    for oesid in oesunits:
        oesunits[oesid]["emu"]["pvc_charge_power"] = round((1-weight)*sol[step_now] + weight*sol[step_next], 2)  # sol[W]
        print("pvc", oesunits)
        
    print("sol[step_now]", sol[step_now], "\n", "sol[step_next]", sol[step_next])
    print("pvc power", round((1-weight)*sol[step_now] + weight*sol[step_next], 2))
    print((1-weight)*sol[step_now] + weight*sol[step_next])
    
    return True

In [3]:
count_s = 525600 * 60 - 3600*9.5   # how many seconds have passed
count_h = float(count_s) / 3600
print("count_h = {:.4f} hours".format(count_h))
oesunits = {}
sol = np.loadtxt('/Users/Huang/Documents/APIS/apis-emulator/data/input/Sample/sample_solar_data.csv', delimiter=',')
print("sol.size: ", sol.size, "\n", "data size is {:.0f} days".format(int(sol.size/24)))

pvc = old_pvcUpdate_Sample()
print(pvc)

count_h = 8750.5000 hours
sol.size:  8760 
 data size is 365 days
weight:  0.5
step_now:  (364, 14)
step_next: (364, 15)
sol[step_now] 472.8 
 sol[step_next] 404.1
pvc power 438.45
438.45000000000005
True


## OUR DATA update, solar rad

In [37]:
def PVCUpdate():
#     count_s = 3600*12 # how many seconds have passed
    count_t = float(count_s)/30  # set counter for data which is collected every 30s
    print(count_t)
    weight = count_t-int(count_t)
    print("weight: ", weight)

    
#     step_now = (int((count_h) / 24)), int((count_h) % 24)
#     print("step_now: ", step_now)
    
#     step_next = (int((count_h+1)/24)), int((count_h+1) % 24)
#     print("step_next:", step_next)
    
    step_now =  (int((count_t) / 2880)+1) * int((count_t) % 2880)
    print("step_now: ", step_now, "\n", "our_data[step_now]", our_data[step_now])

    step_next = (int((count_t+1) / 2880)+1) * int((count_t+1) % 2880)
    print("step_next:", step_next, "\n", "our_data[step_next]", our_data[step_next])

    if int(count_t+1) >= our_data.size:
        logger.debug("no more our_data radiation data")
        return False
    for oesid in oesunits:
        oesunits[oesid]["emu"]["pvc_charge_power"] = round((1-weight)*our_data[step_now] + weight*our_data[step_next], 2)  # our_data[W]
        print("pvc", oesunits)
        
    print("our_data[step_now]", our_data[step_now], "\n", "our_data[step_next]", our_data[step_next])
    print("pvc power", round((1-weight)*our_data[step_now] + weight*our_data[step_next], 2))
    print((1-weight)*our_data[step_now] + weight*our_data[step_next])
    
    return True
    

In [78]:
# test for one day raw data (every 30s)
col = 4
# our_data = np.loadtxt('/Users/Huang/Documents/DQNBattery/data/house212_2019_apis_all.csv', delimiter=',', skiprows=1, usecols=col)
# print(type(our_data))
our_data_all = pd.read_csv('/Users/Huang/Documents/DQNBattery/data/house212_2019_apis_all.csv')#, delimiter=',', skiprows=1, usecols=col)
our_data_all = our_data_all.fillna(method="ffill", inplace=False)
our_data = our_data_all['pvc_charge_power'].values

# data_h212 = np.loadtxt("/Users/Huang/Documents/DQNBattery/data/house212_2019_sift/house212_20190101.csv")
print(our_data, "data size is", our_data.size, type(our_data))  # 0~2879
print(our_data[1566])
# pvc_212 = data_h212[""]
# plt.plot(our_data)
# plt.show()

count_s = 3600*24 - 30*1000.5 # how many seconds have passed
# count_t = float(count_s)/30  # set counter for data which is collected every 30s
# print(count_t)
# weight = count_t-int(count_t)
# print("weight: ", weight)

oesunits = {}

pvc = PVCUpdate()
print(pvc)


[0. 0. 0. ... 0. 0. 0.] data size is 956160 <class 'numpy.ndarray'>
788.0
1879.5
weight:  0.5
step_now:  1879 
 our_data[step_now] 255.0
step_next: 1880 
 our_data[step_next] 233.0
our_data[step_now] 255.0 
 our_data[step_next] 233.0
pvc power 244.0
244.0
True


## Our data updates, load

In [80]:
def LOADUpdate():
#     count_s = 3600*12 # how many seconds have passed
    count_t = float(count_s)/30  # set counter for data which is collected every 30s
    print(count_t)
    weight = count_t-int(count_t)
    print("weight: ", weight)

    step_now =  (int((count_t) / 2880)+1) * int((count_t) % 2880)
    print("step_now: ", step_now, "\n", "our_data[step_now]", our_data[step_now])

    step_next = (int((count_t+1) / 2880)+1) * int((count_t+1) % 2880)
    print("step_next:", step_next, "\n", "our_data[step_next]", our_data[step_next])

    if int(count_t+1) >= our_data.size:
        logger.debug("no more our_data load data")
        return False
    for oesid in oesunits:
        oesunits[oesid]["emu"]["ups_output_power"] = round((1-weight)*our_data[step_now] + weight*our_data[step_next], 2)  # our_data[W]
        print("load", oesunits)
        
    print("our_data[step_now]", our_data[step_now], "\n", "our_data[step_next]", our_data[step_next])
    print("pvc power", round((1-weight)*our_data[step_now] + weight*our_data[step_next], 2))
    print((1-weight)*our_data[step_now] + weight*our_data[step_next])
    
    return True
    

In [81]:
# test for one day raw data (every 30s)

our_data_all = pd.read_csv('/Users/Huang/Documents/DQNBattery/data/house212_2019_apis_all.csv')
our_data_all = our_data_all.fillna(method="ffill", inplace=False)
our_data = our_data_all['ups_output_power'].values

# data_h212 = np.loadtxt("/Users/Huang/Documents/DQNBattery/data/house212_2019_sift/house212_20190101.csv")
print(our_data, "data size is", our_data.size, type(our_data))  # 0~2879
print(our_data[1566])
# pvc_212 = data_h212[""]
# plt.plot(our_data)
# plt.show()

count_s = 3600*24 - 30*1000.5 # how many seconds have passed
# count_t = float(count_s)/30  # set counter for data which is collected every 30s
# print(count_t)
# weight = count_t-int(count_t)
# print("weight: ", weight)

oesunits = {}

load = LOADUpdate()
print(load)


[158. 152. 155. ...   0.   0.   0.] data size is 956160 <class 'numpy.ndarray'>
147.0
1879.5
weight:  0.5
step_now:  1879 
 our_data[step_now] 116.0
step_next: 1880 
 our_data[step_next] 115.0
our_data[step_now] 116.0 
 our_data[step_next] 115.0
pvc power 115.5
115.5
True
